In [1]:
# imports / db pointers

from pymongo import MongoClient
from pprint import pprint
client = MongoClient("mongodb://takuu.dcs.gla.ac.uk:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false")
db = client.Game_data



In [2]:
# server status
serverStatusResult=db.command("serverStatus")
pprint(serverStatusResult)


{'asserts': {'msg': 0, 'regular': 0, 'rollovers': 0, 'user': 0, 'warning': 0},
 'connections': {'available': 809, 'current': 10, 'totalCreated': 1071},
 'extra_info': {'note': 'fields vary by platform', 'page_faults': 7},
 'globalLock': {'activeClients': {'readers': 0, 'total': 19, 'writers': 0},
                'currentQueue': {'readers': 0, 'total': 0, 'writers': 0},
                'totalTime': 2331027388000},
 'host': 'takuudcsglaacuk',
 'localTime': datetime.datetime(2020, 4, 5, 17, 9, 31, 591000),
 'locks': {'Collection': {'acquireCount': {'r': 4372565, 'w': 74750}},
           'Database': {'acquireCount': {'R': 62,
                                         'W': 7788,
                                         'r': 4372591,
                                         'w': 74749},
                        'acquireWaitCount': {'r': 1},
                        'timeAcquiringMicros': {'r': 40745}},
           'Global': {'acquireCount': {'W': 6, 'r': 11158888, 'w': 82537}}},
 'logicalSession

In [3]:
import re

# Parse the lookup tables
chars = ["K","A","W","R","H","M","B","G"]
pairs = []
for i in range(8):
    for j in range(i+1,8):
        pairs += [chars[i]+chars[j]]

lookup = {}        
        
        
for p in pairs:
    with open("C:/Users/bkav9/Documents/Python Scripts/RPGLite/lookup_tables/" + p + "_optimal_moves.txt","r") as f:
        single_pair_dict = {}
        for line in f.readlines()[2:]:
            state = re.split("\t+",line)[:-2]
            action = re.split("\t+",line)[-1][3:-1]
            if action in single_pair_dict.keys():
                single_pair_dict[action] += [state]
            else:
                single_pair_dict[action] = [state]
        lookup[p] = single_pair_dict
        print("finished: " + p)

finished: KA
finished: KW
finished: KR
finished: KH
finished: KM
finished: KB
finished: KG
finished: AW
finished: AR
finished: AH
finished: AM
finished: AB
finished: AG
finished: WR
finished: WH
finished: WM
finished: WB
finished: WG
finished: RH
finished: RM
finished: RB
finished: RG
finished: HM
finished: HB
finished: HG
finished: MB
finished: MG
finished: BG


In [4]:
import numpy as np

# read in the constants

KNIGHT_HEALTH = 10
KNIGHT_DAMAGE = 4
KNIGHT_ACCURACY = 60

ARCHER_HEALTH = 8
ARCHER_DAMAGE = 2
ARCHER_ACCURACY = 85

HEALER_HEALTH = 10
HEALER_DAMAGE = 2
HEALER_ACCURACY = 85
HEALER_HEAL = 1

ROGUE_HEALTH = 8
ROGUE_DAMAGE = 3
ROGUE_ACCURACY = 75
ROGUE_EXECUTE = 5

WIZARD_HEALTH = 8
WIZARD_DAMAGE = 2
WIZARD_ACCURACY = 85

MONK_HEALTH = 7
MONK_DAMAGE = 1
MONK_ACCURACY = 80

GUNNER_HEALTH = 8
GUNNER_DAMAGE = 4
GUNNER_MISS_DAMAGE = 1
GUNNER_ACCURACY = 75

BARBARIAN_HEALTH = 10
BARBARIAN_DAMAGE = 3
BARBARIAN_RAGE_DAMAGE = 5
BARBARIAN_RAGE_THRESHOLD = 4
BARBARIAN_ACCURACY = 75


# Helper methods to build state and parse transitions
def set_health(char):
    
    if char == "Knight":
        return KNIGHT_HEALTH
    if char == "Archer":        
        return ARCHER_HEALTH
    if char == "Healer":        
        return HEALER_HEALTH
    if char == "Rogue":        
        return ROGUE_HEALTH
    if char == "Wizard":        
        return WIZARD_HEALTH
    if char == "Monk":        
        return MONK_HEALTH
    if char == "Gunner":        
        return GUNNER_HEALTH
    if char == "Barbarian":        
        return BARBARIAN_HEALTH
   

# Take an action string and state [], update the state
def do_action(action, state):
    if "skip" in action: 
        return [0]*8 + ["-"] + [0]*8 + ["-"] # no change.
    if action[2] == "K":
        do_knight_action(action, state)
    if action[2] == "A":
        do_archer_action(action, state)
    if action[2] == "H":
        do_healer_action(action, state)
    if action[2] == "R":
        do_rogue_action(action, state)
    if action[2] == "W":
        do_wizard_action(action, state)
    if action[2] == "M":
        do_monk_action(action, state)
    if action[2] == "G":
        do_gunner_action(action, state)
    if action[2] == "B":
        do_barbarian_action(action, state)
    # Ensure min health is 0
    Is = [0,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16]
    for i in Is:
        state[i] = max(0,state[i])
    # Reset stun
    if action[1] == "1":
        state[8] = "-"
    else:
        state[17] = "-"

def do_knight_action(action, state):
    # action e.g.: p1Kp2H_41
    roll = int(action.split("_")[1])
    if roll >= 100 - KNIGHT_ACCURACY:
        if action[1] == "1":
            state[9+chars.index(action[5])] -= KNIGHT_DAMAGE
        else:
            state[chars.index(action[5])] -= KNIGHT_DAMAGE
        
def do_archer_action(action, state):
    # action e.g.: p2Ap1K_55p1R_86 OR p2Ap1K_53
    roll1 = int(action.split("_")[1].split("p")[0])
    if roll1 >= 100 - ARCHER_ACCURACY:
        if action[1] == "1":
            state[9+chars.index(action[5])] -= ARCHER_DAMAGE
        else:
            state[chars.index(action[5])] -= ARCHER_DAMAGE
    # If there is no second target
    if len(action.split("_")) == 2:
        return
    roll2 = int(action.split("_")[2])
    target = action.split("_")[1][-1]
    if roll1 >= 100 - ARCHER_ACCURACY:
        if action[1] == "1":
            state[9+chars.index(target)] -= ARCHER_DAMAGE
        else:
            state[chars.index(target)] -= ARCHER_DAMAGE

def do_healer_action(action, state):
    # action e.g.: p2Hp1Rp2H_4
    roll = int(action.split("_")[1])
    if roll >= 100 - HEALER_ACCURACY:
        if action[1] == "1":
            state[9+chars.index(action[5])] -= HEALER_DAMAGE
            state[chars.index("H")] += HEALER_HEAL
        else:
            state[chars.index(action[5])] -= HEALER_DAMAGE
            state[9+chars.index("H")] += HEALER_HEAL

def do_rogue_action(action, state):
    # action e.g.: p1Rp2H_53
    roll = int(action.split("_")[1])
    if roll >= 100 - KNIGHT_ACCURACY:
        if state[chars.index(action[5])] <= ROGUE_EXECUTE:
            # Finish him.
            if action[1] == "1":
                state[9+chars.index(action[5])] = 0
            else:
                state[chars.index(action[5])] = 0
        else:
            if action[1] == "1":
                state[9+chars.index(action[5])] -= ROGUE_DAMAGE
            else:
                state[chars.index(action[5])] -= ROGUE_DAMAGE

def do_wizard_action(action, state):
    # action e.g.: p1Wp2W_13
    roll = int(action.split("_")[1])
    if roll >= 100 - WIZARD_ACCURACY:
        if action[1] == "1":
            state[9+chars.index(action[5])] -= WIZARD_DAMAGE
            state[17] = action[5]
        else:
            state[chars.index(action[5])] -= WIZARD_DAMAGE
            state[8] = action[5]

def do_monk_action(action, state):
    # etc
    roll = int(action.split("_")[1])
    if roll >= 100 - MONK_ACCURACY:
        if action[1] == "1":
            state[9+chars.index(action[5])] -= MONK_DAMAGE
        else:
            state[chars.index(action[5])] -= MONK_DAMAGE
            
def do_gunner_action(action, state):
    roll = int(action.split("_")[1])
    if roll >= 100 - GUNNER_ACCURACY:
        if action[1] == "1":
            state[9+chars.index(action[5])] -= GUNNER_DAMAGE
        else:
            state[chars.index(action[5])] -= GUNNER_DAMAGE
    else:
        if action[1] == "1":
            state[9+chars.index(action[5])] -= GUNNER_MISS_DAMAGE
        else:
            state[chars.index(action[5])] -= GUNNER_MISS_DAMAGE

def do_barbarian_action(action, state):
    roll = int(action.split("_")[1])
    if roll >= 100 - BARBARIAN_ACCURACY:
        if action[1] == "1":
            if int(state[7]) <= BARBARIAN_RAGE_THRESHOLD:
                state[9+chars.index(action[5])] -= BARBARIAN_RAGE_DAMAGE
            else:
                state[9+chars.index(action[5])] -= BARBARIAN_DAMAGE
        else:
            if int(state[15]) <= BARBARIAN_RAGE_THRESHOLD:
                state[chars.index(action[5])] -= BARBARIAN_RAGE_DAMAGE
            else:
                state[chars.index(action[5])] -= BARBARIAN_DAMAGE
            

In [7]:
# check if the move played was optimal
def move_lookup(state, move, pair):
    global corr, incorr
    lookup_state = [0]*11
    # get the state in the right form
    if move[1] == "1":
        lookup_state[0] = state[chars.index(pair[0])]
        lookup_state[1] = state[chars.index(pair[1])]
        lookup_state[2] = state[8]
        lookup_state[3:] = state[9:]
    else:
        print(9+chars.index(pair[0]))
        lookup_state[0] = state[9+chars.index(pair[0])]
        lookup_state[1] = state[9+chars.index(pair[1])]
        lookup_state[2] = state[17]
        lookup_state[3:] = state[:9]
    
    # get the move in the right form
    move_notation = ""
    for letter in move:
        if letter in chars:
            move_notation += letter
        if len(move_notation) == 1:
            move_notation += "_"
    if len(move_notation) > 3 and move_notation[0] == "A":
        if chars.index(move_notation[2]) > chars.index(move_notation[3]):
            tmp = move_notation[:2] + move_notation[3] + move_notation[2]
            move_notation = tmp
    found = False
    tmp = []
    for letter in lookup_state:
        tmp += [str(letter)]
    lookup_state = tmp
    for k in lookup[pair].keys():
        if lookup_state in lookup[pair][k]:
            found = True
            if k == move_notation:
                print("OPTIMAL MOVE PLAYED")
                corr+=1
            else:
                print("MISTAKE:" + k + ", is the correct move")
                incorr+=1
    if not found:
        print(lookup_state, "not found")


In [8]:
# open all games and calculates how many 
      
count = 1
malformed = 0
corr = 0                 # optimal moves made
incorr = 0               # suboptimal moves made
moves = 0

games = db.completed_games
for g in games.find():
    if not "p1c1" in g or not "p2c1" in g or len(g["Moves"]) < 2:
        malformed += 1
        continue
    print("Game", str(count) + ":", g["p1c1"], g["p1c2"], "vs", g["p2c1"], g["p2c2"])
    team_1 = g["p1c1"][0] + g["p1c2"][0]
    if chars.index(team_1[0]) > chars.index(team_1[1]):
        team_1 = g["p1c2"][0] + g["p1c1"][0]
    
    team_2 = g["p2c1"][0] + g["p2c2"][0]
    if chars.index(team_2[0]) > chars.index(team_2[1]):
        team_2 = g["p2c2"][0] + g["p2c1"][0]
    state = [0]*8 + ["-"] + [0]*8 + ["-"]
    state[chars.index(g["p1c1"][0])] = set_health(g["p1c1"])
    state[chars.index(g["p1c2"][0])] = set_health(g["p1c2"])
    state[chars.index(g["p2c1"][0]) + 9] = set_health(g["p2c1"])
    state[chars.index(g["p2c2"][0]) + 9] = set_health(g["p2c2"])
    move = 1
    for m in g["Moves"]:
        moves+=1
        print("\tstate = " + str(state))
        print("\tmove " + str(move) + ": " + m)
        if m[1] == "1":
            move_lookup(state, m, team_1)
        else:
            move_lookup(state, m, team_2)
        do_action(m, state)
        move+=1
    print("~~\n" + str(state)+"\n")
    count+=1
    
print("malformed:",malformed)

print("Games: " + str(count))
print("Moves: " + str(moves))
print("optimal: " + str(corr))
print("suboptimal: " + str(incorr))

Game 1: Archer Knight vs Knight Rogue
	state = [10, 8, 0, 0, 0, 0, 0, 0, '-', 10, 0, 0, 8, 0, 0, 0, 0, '-']
	move 1: p2Kp1A_73
9
MISTAKE:R_A, is the correct move
	state = [10, 4, 0, 0, 0, 0, 0, 0, '-', 10, 0, 0, 8, 0, 0, 0, 0, '-']
	move 2: p1Ap2K_21p2R_95
OPTIMAL MOVE PLAYED
	state = [10, 4, 0, 0, 0, 0, 0, 0, '-', 8, 0, 0, 6, 0, 0, 0, 0, '-']
	move 3: p2Rp1A_64
9
MISTAKE:R_Ae, is the correct move
	state = [10, 0, 0, 0, 0, 0, 0, 0, '-', 8, 0, 0, 6, 0, 0, 0, 0, '-']
	move 4: p1Kp2R_77
OPTIMAL MOVE PLAYED
	state = [10, 0, 0, 0, 0, 0, 0, 0, '-', 8, 0, 0, 2, 0, 0, 0, 0, '-']
	move 5: p2Kp1K_18
9
MISTAKE:R_K, is the correct move
	state = [10, 0, 0, 0, 0, 0, 0, 0, '-', 8, 0, 0, 2, 0, 0, 0, 0, '-']
	move 6: p1Kp2R_66
OPTIMAL MOVE PLAYED
	state = [10, 0, 0, 0, 0, 0, 0, 0, '-', 8, 0, 0, 0, 0, 0, 0, 0, '-']
	move 7: p2Kp1K_1
9
OPTIMAL MOVE PLAYED
	state = [10, 0, 0, 0, 0, 0, 0, 0, '-', 8, 0, 0, 0, 0, 0, 0, 0, '-']
	move 8: p1Kp2K_33
OPTIMAL MOVE PLAYED
	state = [10, 0, 0, 0, 0, 0, 0, 0, '-', 8, 

	state = [0, 0, 0, 6, 0, 0, 0, 0, '-', 0, 0, 0, 0, 9, 0, 0, 0, '-']
	move 8: p1Rp2K_22
MISTAKE:R_H, is the correct move
	state = [0, 0, 0, 6, 0, 0, 0, 0, '-', 0, 0, 0, 0, 9, 0, 0, 0, '-']
	move 9: p2Kp1R_14
9
MISTAKE:H_RH, is the correct move
	state = [0, 0, 0, 6, 0, 0, 0, 0, '-', 0, 0, 0, 0, 9, 0, 0, 0, '-']
	move 10: p1Rp2K_10
MISTAKE:R_H, is the correct move
	state = [0, 0, 0, 6, 0, 0, 0, 0, '-', 0, 0, 0, 0, 9, 0, 0, 0, '-']
	move 11: p2Kp1R_93
9
MISTAKE:H_RH, is the correct move
	state = [0, 0, 0, 2, 0, 0, 0, 0, '-', 0, 0, 0, 0, 9, 0, 0, 0, '-']
	move 12: p1Rp2K_81
MISTAKE:R_H, is the correct move
	state = [0, 0, 0, 2, 0, 0, 0, 0, '-', 0, 0, 0, 0, 9, 0, 0, 0, '-']
	move 13: p2Hp1Rp2H_43
9
OPTIMAL MOVE PLAYED
~~
[0, 0, 0, 0, 0, 0, 0, 0, '-', 0, 0, 0, 0, 10, 0, 0, 0, '-']

Game 8: Wizard Rogue vs Knight Archer
	state = [0, 0, 8, 8, 0, 0, 0, 0, '-', 10, 8, 0, 0, 0, 0, 0, 0, '-']
	move 1: p2Ap1W_14p1R_46
9
OPTIMAL MOVE PLAYED
	state = [0, 0, 8, 8, 0, 0, 0, 0, '-', 10, 8, 0, 0, 0, 0, 0,

	state = [0, 0, 0, 0, 11, 0, 0, 0, '-', 0, 0, 0, 0, 10, 0, 0, 0, '-']
	move 9: p2Hp1Hp2K_79
9
['0', '10', '-', '0', '0', '0', '0', '11', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 0, 9, 0, 0, 0, '-', 0, 0, 0, 0, 11, 0, 0, 0, '-']
	move 10: p1Hp2Kp1H_27
['0', '9', '-', '0', '0', '0', '0', '11', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 0, 10, 0, 0, 0, '-', 0, 0, 0, 0, 11, 0, 0, 0, '-']
	move 11: p2Kp1H_3
9
['0', '11', '-', '0', '0', '0', '0', '10', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 0, 10, 0, 0, 0, '-', 0, 0, 0, 0, 11, 0, 0, 0, '-']
	move 12: p1Hp2Kp1H_30
['0', '10', '-', '0', '0', '0', '0', '11', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 0, 11, 0, 0, 0, '-', 0, 0, 0, 0, 11, 0, 0, 0, '-']
	move 13: p2Hp1H_69
9
['0', '11', '-', '0', '0', '0', '0', '11', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 0, 9, 0, 0, 0, '-', 0, 0, 0, 0, 12, 0, 0, 0, '-']
	move 14: p1Hp2Hp1H_74
['0', '9', '-', '0', '0', '0', '0', '12', '0', '0', '0', '-'] not found
	state = [

	state = [0, 0, 4, 6, 0, 0, 0, 0, '-', 0, 6, 0, 0, 0, 0, 0, 0, '-']
	move 5: p2Ap1W_96p1R_84
10
OPTIMAL MOVE PLAYED
	state = [0, 0, 2, 4, 0, 0, 0, 0, '-', 0, 6, 0, 0, 0, 0, 0, 0, '-']
	move 6: p1Rp2A_37
MISTAKE:W_A, is the correct move
	state = [0, 0, 2, 4, 0, 0, 0, 0, '-', 0, 6, 0, 0, 0, 0, 0, 0, '-']
	move 7: p2Ap1W_93p1R_49
10
OPTIMAL MOVE PLAYED
	state = [0, 0, 0, 2, 0, 0, 0, 0, '-', 0, 6, 0, 0, 0, 0, 0, 0, '-']
	move 8: p1Rp2A_45
OPTIMAL MOVE PLAYED
	state = [0, 0, 0, 2, 0, 0, 0, 0, '-', 0, 0, 0, 0, 0, 0, 0, 0, '-']
	move 9: p2Wp1R_98
10
['0', '0', '-', '0', '0', '0', '2', '0', '0', '0', '0', '-'] not found
~~
[0, 0, 0, 0, 0, 0, 0, 0, 'R', 0, 0, 0, 0, 0, 0, 0, 0, '-']

Game 18: Knight Rogue vs Archer Healer
	state = [10, 0, 0, 8, 0, 0, 0, 0, '-', 0, 8, 0, 0, 10, 0, 0, 0, '-']
	move 1: p2Ap1K_2p1R_90
10
OPTIMAL MOVE PLAYED
	state = [10, 0, 0, 8, 0, 0, 0, 0, '-', 0, 8, 0, 0, 10, 0, 0, 0, '-']
	move 2: p1Kp2A_85
OPTIMAL MOVE PLAYED
	state = [10, 0, 0, 8, 0, 0, 0, 0, '-', 0, 4, 0, 0, 

['0', '9', '-', '0', '0', '0', '6', '11', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 4, 11, 0, 0, 0, '-', 0, 0, 0, 0, 10, 0, 0, 0, '-']
	move 7: p1Rp2H_1
['4', '11', '-', '0', '0', '0', '0', '10', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 4, 11, 0, 0, 0, '-', 0, 0, 0, 0, 10, 0, 0, 0, '-']
	move 8: p2Hp1Rp2H_97
11
['0', '10', '-', '0', '0', '0', '4', '11', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 2, 11, 0, 0, 0, '-', 0, 0, 0, 0, 11, 0, 0, 0, '-']
	move 9: p1Hp2Hp1R_99
['2', '11', '-', '0', '0', '0', '0', '11', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 2, 12, 0, 0, 0, '-', 0, 0, 0, 0, 9, 0, 0, 0, '-']
	move 10: p2Hp1Rp2H_3
11
['0', '9', '-', '0', '0', '0', '2', '12', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 2, 12, 0, 0, 0, '-', 0, 0, 0, 0, 9, 0, 0, 0, '-']
	move 11: p1Hp2Hp1R_76
['2', '12', '-', '0', '0', '0', '0', '9', '0', '0', '0', '-'] not found
	state = [0, 0, 0, 2, 13, 0, 0, 0, '-', 0, 0, 0, 0, 7, 0, 0, 0, '-']
	move 12: p2Hp1Rp2H_42
11
['0', '7'

...

OPTIMAL MOVE PLAYED
	state = [0, 0, 8, 8, 0, 0, 0, 0, '-', 0, 0, 5, 8, 0, 0, 0, 0, '-']
	move 3: p1Rp2W_61
MISTAKE:R_We, is the correct move
	state = [0, 0, 8, 8, 0, 0, 0, 0, '-', 0, 0, 2, 8, 0, 0, 0, 0, '-']
	move 4: p2Rp1W_76
11
OPTIMAL MOVE PLAYED
	state = [0, 0, 5, 8, 0, 0, 0, 0, '-', 0, 0, 2, 8, 0, 0, 0, 0, '-']
	move 5: p1Wp2R_13
MISTAKE:W_W, is the correct move
	state = [0, 0, 5, 8, 0, 0, 0, 0, '-', 0, 0, 2, 8, 0, 0, 0, 0, '-']
	move 6: p2Rp1W_26
11
MISTAKE:R_We, is the correct move
	state = [0, 0, 5, 8, 0, 0, 0, 0, '-', 0, 0, 2, 8, 0, 0, 0, 0, '-']
	move 7: p1Rp2R_29
MISTAKE:W_W, is the correct move
	state = [0, 0, 5, 8, 0, 0, 0, 0, '-', 0, 0, 2, 8, 0, 0, 0, 0, '-']
	move 8: p2Rp1R_44
11
MISTAKE:R_We, is the correct move
	state = [0, 0, 5, 5, 0, 0, 0, 0, '-', 0, 0, 2, 8, 0, 0, 0, 0, '-']
	move 9: p1Rp2R_26
MISTAKE:W_R, is the correct move
~~
[0, 0, 5, 5, 0, 0, 0, 0, '-', 0, 0, 2, 8, 0, 0, 0, 0, '-']

Game 41: Monk Healer vs Knight Rogue
	state = [0, 0, 0, 0, 10, 7, 0, 0, '-', 1

Game 47: Rogue Archer vs Healer Archer
	state = [0, 8, 0, 8, 0, 0, 0, 0, '-', 0, 8, 0, 0, 10, 0, 0, 0, '-']
	move 1: p2Ap1R_57p1A_32
10
OPTIMAL MOVE PLAYED
	state = [0, 6, 0, 6, 0, 0, 0, 0, '-', 0, 8, 0, 0, 10, 0, 0, 0, '-']
	move 2: p1Ap2H_50
MISTAKE:A_AH, is the correct move
	state = [0, 6, 0, 6, 0, 0, 0, 0, '-', 0, 8, 0, 0, 8, 0, 0, 0, '-']
	move 3: p2Ap1R_90p1A_60
10
OPTIMAL MOVE PLAYED
~~
[0, 4, 0, 4, 0, 0, 0, 0, '-', 0, 8, 0, 0, 8, 0, 0, 0, '-']

Game 48: Barbarian Archer vs Knight Barbarian
	state = [0, 8, 0, 0, 0, 0, 10, 0, '-', 10, 0, 0, 0, 0, 0, 10, 0, '-']
	move 1: p1Ap2K_39p2B_10
OPTIMAL MOVE PLAYED
	state = [0, 8, 0, 0, 0, 0, 10, 0, '-', 8, 0, 0, 0, 0, 0, 8, 0, '-']
	move 2: p2Kp1A_32
9
MISTAKE:B_B, is the correct move
	state = [0, 8, 0, 0, 0, 0, 10, 0, '-', 8, 0, 0, 0, 0, 0, 8, 0, '-']
	move 3: p1Ap2K_56p2B_45
OPTIMAL MOVE PLAYED
	state = [0, 8, 0, 0, 0, 0, 10, 0, '-', 6, 0, 0, 0, 0, 0, 6, 0, '-']
	move 4: p2Kp1A_0
9
OPTIMAL MOVE PLAYED
	state = [0, 8, 0, 0, 0, 0, 10, 0, 

OPTIMAL MOVE PLAYED
~~
[10, 0, 0, 0, 0, 1, 0, 0, '-', 0, 0, 0, 0, 0, 0, 0, 0, '-']

Game 56: Archer Gunner vs Rogue Knight
	state = [0, 8, 0, 0, 0, 0, 0, 8, '-', 10, 0, 0, 8, 0, 0, 0, 0, '-']
	move 1: p2Rp1G_67
9
OPTIMAL MOVE PLAYED
	state = [0, 8, 0, 0, 0, 0, 0, 5, '-', 10, 0, 0, 8, 0, 0, 0, 0, '-']
	move 2: p1Ap2R_8p2K_14
OPTIMAL MOVE PLAYED
	state = [0, 8, 0, 0, 0, 0, 0, 5, '-', 10, 0, 0, 8, 0, 0, 0, 0, '-']
	move 3: p2Rp1G_43
9
MISTAKE:R_Ge, is the correct move
	state = [0, 8, 0, 0, 0, 0, 0, 0, '-', 10, 0, 0, 8, 0, 0, 0, 0, '-']
	move 4: p1Ap2R_51p2K_20
OPTIMAL MOVE PLAYED
	state = [0, 8, 0, 0, 0, 0, 0, 0, '-', 8, 0, 0, 6, 0, 0, 0, 0, '-']
	move 5: p2Kp1A_26
9
MISTAKE:R_A, is the correct move
	state = [0, 8, 0, 0, 0, 0, 0, 0, '-', 8, 0, 0, 6, 0, 0, 0, 0, '-']
	move 6: p1Ap2R_78p2K_90
OPTIMAL MOVE PLAYED
	state = [0, 8, 0, 0, 0, 0, 0, 0, '-', 6, 0, 0, 4, 0, 0, 0, 0, '-']
	move 7: p2Kp1A_75
9
OPTIMAL MOVE PLAYED
	state = [0, 4, 0, 0, 0, 0, 0, 0, '-', 6, 0, 0, 4, 0, 0, 0, 0, '-']
	mov

	state = [0, 8, 0, 0, 0, 0, 0, 0, '-', 0, 0, 0, 6, 0, 0, 0, 0, '-']
	move 5: p1Ap2R_70
OPTIMAL MOVE PLAYED
	state = [0, 8, 0, 0, 0, 0, 0, 0, '-', 0, 0, 0, 4, 0, 0, 0, 0, '-']
	move 6: p2Kp1A_80
9
MISTAKE:R_A, is the correct move
	state = [0, 4, 0, 0, 0, 0, 0, 0, '-', 0, 0, 0, 4, 0, 0, 0, 0, '-']
	move 7: p1Ap2R_56
OPTIMAL MOVE PLAYED
	state = [0, 4, 0, 0, 0, 0, 0, 0, '-', 0, 0, 0, 2, 0, 0, 0, 0, '-']
	move 8: p2Rp1A_55
9
MISTAKE:R_Ae, is the correct move
~~
[0, 0, 0, 0, 0, 0, 0, 0, '-', 0, 0, 0, 2, 0, 0, 0, 0, '-']

Game 66: Knight Archer vs Healer Archer
	state = [10, 8, 0, 0, 0, 0, 0, 0, '-', 0, 8, 0, 0, 10, 0, 0, 0, '-']
	move 1: p1Ap2H_79p2A_2
OPTIMAL MOVE PLAYED
	state = [10, 8, 0, 0, 0, 0, 0, 0, '-', 0, 6, 0, 0, 8, 0, 0, 0, '-']
	move 2: p2Ap1K_60p1A_35
10
OPTIMAL MOVE PLAYED
	state = [8, 6, 0, 0, 0, 0, 0, 0, '-', 0, 6, 0, 0, 8, 0, 0, 0, '-']
	move 3: p1Ap2H_47p2A_95
OPTIMAL MOVE PLAYED
	state = [8, 6, 0, 0, 0, 0, 0, 0, '-', 0, 4, 0, 0, 6, 0, 0, 0, '-']
	move 4: p2Hp1Ap2A_23
10
M

['4', '4', '-', '8', '0', '0', '0', '0', '0', '0', '0', 'K'] not found
	state = [6, 0, 0, 0, 0, 0, 0, 0, 'K', 0, 0, 4, 0, 0, 0, 0, 4, '-']
	move 9: p1_skip
MISTAKE:skip, is the correct move
	state = [6, 0, 0, 0, 0, 0, 0, 0, 'K', 0, 0, 4, 0, 0, 0, 0, 4, '-']
	move 10: p2Wp1K_60
11
['4', '4', '-', '6', '0', '0', '0', '0', '0', '0', '0', 'K'] not found
	state = [4, 0, 0, 0, 0, 0, 0, 0, 'K', 0, 0, 4, 0, 0, 0, 0, 4, '-']
	move 11: p1_skip
MISTAKE:skip, is the correct move
	state = [4, 0, 0, 0, 0, 0, 0, 0, 'K', 0, 0, 4, 0, 0, 0, 0, 4, '-']
	move 12: p2Gp1K_49
11
['4', '4', '-', '4', '0', '0', '0', '0', '0', '0', '0', 'K'] not found
~~
[0, 0, 0, 0, 0, 0, 0, 0, 'K', 0, 0, 4, 0, 0, 0, 0, 4, '-']

Game 73: Gunner Rogue vs Archer Healer
	state = [0, 0, 0, 8, 0, 0, 0, 8, '-', 0, 8, 0, 0, 10, 0, 0, 0, '-']
	move 1: p2Ap1G_45p1R_17
10
OPTIMAL MOVE PLAYED
	state = [0, 0, 0, 6, 0, 0, 0, 6, '-', 0, 8, 0, 0, 10, 0, 0, 0, '-']
	move 2: p1Gp2A_1
OPTIMAL MOVE PLAYED
	state = [0, 0, 0, 6, 0, 0, 0, 6, '-', 0

In [120]:
a_string = "some text"
print(a_string[2:])

me text


In [121]:
print(a_string[:2])

so
